In [1]:
import os

In [2]:
os.chdir("../")

In [4]:
%pwd

'e:\\Projects\\E2E Emotion Detection from text\\Emotion-Detection-using-ML'

In [16]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path_X: Path
    train_data_path_target: Path
    test_data_path_X: Path
    test_data_path_target: Path
    model_name: str
    
    # model
    class_weight: str
    max_iter: int
    penalty: str
    solver: str
    n_jobs: int
    random_state: int
    
    # target_column:str    

In [17]:
from src.ML_emotion_detection.constants import *
from src.ML_emotion_detection.utils.common import read_yaml, create_directories, load_bin, save_bin

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        # schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            
            train_data_path_X = config.train_data_path_X,
            train_data_path_target = config.train_data_path_target,
            
            test_data_path_X = config.test_data_path_X,
            test_data_path_target = config.test_data_path_target,
            
            model_name = config.model_name,
            
            class_weight= params.class_weight,
            max_iter= params.max_iter,
            penalty= params.penalty,
            solver= params.solver,
            n_jobs= params.n_jobs,
            random_state= params.random_state
            
            # target_column = schema.name
            
        )
        return model_trainer_config

In [19]:
from sklearn.linear_model import LogisticRegression

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_x = load_bin(self.config.train_data_path_X)
        test_x = load_bin(self.config.test_data_path_X)
        
        train_y = load_bin(self.config.train_data_path_target)
        test_y = load_bin(self.config.test_data_path_target)

        log_reg = LogisticRegression(class_weight= self.config.class_weight,
                                     max_iter = self.config.max_iter,
                                     penalty = self.config.penalty,
                                     solver = self.config.solver,
                                     n_jobs = self.config.n_jobs,
                                     random_state = self.config.random_state)
        log_reg.fit(train_x, train_y)

        save_bin(log_reg, os.path.join(self.config.root_dir, self.config.model_name))

In [21]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-11-21 10:08:22,537: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-21 10:08:22,558: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-21 10:08:22,563: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-21 10:08:22,566: INFO: common: created directory at: artifacts]


[2024-11-21 10:08:22,570: INFO: common: created directory at: artifacts/model_trainer]
[2024-11-21 10:08:22,724: INFO: common: binary file loaded from: artifacts/data_transformation/X_train.joblib]
[2024-11-21 10:08:22,970: INFO: common: binary file loaded from: artifacts/data_transformation/y_test.joblib]
[2024-11-21 10:08:23,173: INFO: common: binary file loaded from: artifacts/data_transformation/y_train.joblib]
[2024-11-21 10:08:23,177: INFO: common: binary file loaded from: artifacts/data_transformation/y_test.joblib]
[2024-11-21 10:09:47,138: INFO: common: binary file saved at: artifacts/model_trainer\log_reg_model.joblib]
